In [1]:
%load_ext autoreload
%autoreload
from tune_and_draw import TuneAndDraw
from fedhybrid_innerloops import FedHybrid
from parameter import TuneParam,TrainParam
from costFunc import QuadraticCostFunc
import matplotlib.pyplot as plt
from datetime import datetime
import random
import numpy as np
import random
import pandas as pd
from scipy.optimize import minimize


In [2]:
def dataset(nclient,n):   
    mean_nsample_param=4
    sigma_nsample_param=2
    prior_param_mean=0
    prior_param_sigma=0.5
    scaling_param_mean=0 #***
    scaling_param_sigma=0.5#**
    ncond_param = 4
    ndim=3
    print('local datasizes')

    if nclient == 8 and n ==354:
        print('efsd')
        ni=[45, 38, 29, 12, 62, 76, 36, 56]
    else:
        p = np.zeros(nclient)
        s = 0
        for i in range(nclient):
            p[i] = random.uniform(0.1, 1)
            s += p[i]
        p = p/s
        print(p)
        ni = p * n
        ni = [int(num) for num in ni]
        ni[-1] = ni[-1] + n - sum(ni) 
        print(ni)
    A = [[] for _ in range(nclient)]
    y = [[] for _ in range(nclient)]

#### define some eprior ####
    x_0 = np.random.normal(prior_param_mean, prior_param_sigma, ndim)
    scaling = np.random.lognormal(scaling_param_mean, scaling_param_sigma, nclient) # how diverse the data is

    #print('condition number of local Hessians')
      
    for i in range(nclient):
        Ai = np.random.rand(ni[i], ndim) * scaling[i]
        #print(i,':',np.linalg.eigh(np.transpose(Ai).dot(Ai))[0][-1])
        sig = np.random.random(1)
        v = np.random.normal(0, sig, ni[i]) 
        yi = Ai.dot(x_0) + v
        yi = yi.reshape([ni[i], 1])
        A[i] = Ai
        y[i] = yi
    return A,y
        



In [3]:
def prepare(nclient,n):
    np.random.seed(2022)
    random.seed(2022) 
    ndim = 3
    A,y = dataset(nclient,n)
    func = QuadraticCostFunc(A, y, gamma=0.01)
    #initial_x = np.random.rand(ndim, 1)
    #if nclient != 8:
    initial_x = np.zeros([ndim,1])
    print('initial_x',initial_x)
    fn_min = minimize(func.global_func, initial_x, tol=1e-30)
    fn_star = fn_min.fun
    tuneAndDraw = TuneAndDraw(nclient, initial_x,setup=1)
    method = FedHybrid(func, fn_star)
    return method,tuneAndDraw

### synchronous synthetic quadratic:

In [ ]:
method,tuneAndDraw = prepare(10,10000)
ns=[0,2,4,6,8,10]
innerIterations = 1
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/1 synchronous synthetic quadratic node 10.pdf"
tuneAndDraw.draw(df,method.train_innerloop,"1 synchronous synthetic quadratic",filename,save = 0,asyn=0,possionBeta=[])

### asynchronous synthetic quadratic:

In [ ]:
method,tuneAndDraw = prepare(10,10000)
ns=[0,2,4,6,8,10]
innerIterations = 1
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/1 asynchronous synthetic quadratic.pdf"

tuneAndDraw.draw(df,method.asynchronousTrain2,"async synthetic quadratic possionBeta=[4, 5, 10, 4, 2, 1, 2, 7, 100, 5]",filename,save=0,asyn=1,possionBeta = [4, 5, 10, 4, 2, 1, 2, 7,100,5])

### innerloops

In [ ]:
method,tuneAndDraw = prepare(10,10000)
ns=[0,2,4,6,8,10]
innerIterations =4
df = tuneAndDraw.readPa(ns,innerIterations)
filename="./sync_and_async/1 synthetic quadratic innerloop 4 before2.pdf"
tuneAndDraw.draw(df,method.train_innerloop_lambdaBefore,"synthetic quadratic innerloop 4",filename,save = 0,asyn=0,possionBeta=[])